In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Загрузка данных

In [65]:
df_prs = pd.read_csv('person.csv').set_index('Unnamed: 0')
df_tr = pd.read_csv('action_train.csv').set_index('Unnamed: 0')
df_tt = pd.read_csv('action_test.csv').set_index('Unnamed: 0')

df_trainn = df_tr.copy()
df_testt = df_tt.copy()
df_personss = df_prs.copy()

C:\Program Files (x86)\pit\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#  df_type1, это там где мало пропусков
char_1	char_2	char_3	char_4	char_5	char_6	char_7	char_8	char_9	

In [66]:
# for person_id

# for action_id
df1_drop = ['char_10', 'action_type']
df_all_drop = ['char_1', 'char_2', 'char_3', 'char_4', 'char_5',
               'char_6', 'char_7', 'char_8', 'char_9']



In [67]:
def separete_data(df, flag=0):
    y = []
    m = []
    d = []
    
    for i in df.date:
        year, month, day = i.split('-')
        y.append(int(year))
        m.append(int(month))
        d.append(int(day))
   
    df['y'] = y
    df['m'] = m
    df['d'] = d
    
    df.drop('date', axis=1, inplace=True)
    return df

In [68]:
# Разделяем action таблицу на df с type1 и dftype2 и удаляем пустые признаки

def separate_table(dataframe, data_person):
    # Локальные df
    df = dataframe.copy()
    df_person = data_person.copy()
    
    # Где мало пропусков
    df_tp1 = df[df.action_type == 'type 1']
    df_tp1.drop(df1_drop, axis=1, inplace=True)
    
    p_id = df_tp1.person_id.to_list()
    df_tp1_p = df_person[df_person.person_id.isin(p_id)]
    
    # Где много пропусков
    df_all = df[df.action_type != 'type 1']
    df_all.drop(df_all_drop, axis=1, inplace=True)
    
    p_id = df_all.person_id.to_list()
    df_all_p = df_person[df_person.person_id.isin(p_id)]

    return df_tp1, df_all, df_tp1_p, df_all_p

# Теперь надо добавить данные из person_id признаки
def concat_tables(df, df_people):
    
    df_tp1, df_all, df_tp1_p, df_all_p = separate_table(df, df_people)
    df_type1 = pd.merge(df_tp1, df_tp1_p, how='inner', on='person_id')
    df_all = pd.merge(df_all, df_all_p, how='inner', on='person_id')

    df_type1.drop('person_id', axis=1, inplace=True)
    df_all.drop('person_id', axis=1, inplace=True)
    
    return df_type1, df_all


In [69]:
from sklearn import preprocessing

# cols_bool.shape[1] + cols_obj.shape[1] + cols_int.shape[1]
def convert_bool(df):
    
    cols_bool = df.select_dtypes(include=['bool'])
    convert = {False: 0,
              True: 1}
    to_fill = {i: convert for i in cols_bool.columns}

    col_o = cols_bool.columns
    df[col_o]= df[col_o].replace(to_fill)
    return df


def convert_obj(df):
    
    cols_obj = df.select_dtypes(include=['O']).copy()
    cols_obj = cols_obj.drop('action_id', axis=1)
    le = preprocessing.LabelEncoder()
    for col in cols_obj.columns:
        res = le.fit_transform(cols_obj[col])
        df[col] = res
    return df


def convert_data(df):
    
    df = convert_bool(df)
    df = convert_obj(df)
    
    return df
    

In [70]:
df_train = separete_data(df_trainn)
df_persons = separete_data(df_personss, 1)

df_type1, df_all = concat_tables(df_train, df_persons)

df_type1 = convert_data(df_type1)
df_all = convert_data(df_all)

df_type1_base = df_type1.copy() 
df_all_base = df_all.copy()

C:\Program Files (x86)\pit\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [58]:
df_type1

,action_id,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,y_y,m_y,d_y
0,act1_9923,22,27,0,0,5,2,2,14,17,...,0,0,0,0,0,0,76,2022,8,26
1,act1_198174,29,2,6,0,5,0,0,12,0,...,0,0,0,0,0,0,0,2022,7,26
2,act1_214090,16,28,7,2,0,2,3,13,0,...,0,0,0,0,0,0,0,2022,7,26
3,act1_230588,11,11,4,2,4,1,1,12,11,...,0,0,0,0,0,0,0,2022,7,26
4,act1_271874,11,27,4,1,5,0,0,14,17,...,0,0,0,0,0,0,0,2022,7,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157610,act1_272254,3,0,6,3,5,0,0,10,8,...,1,0,1,0,1,0,50,2020,7,21
157611,act1_204921,3,30,9,3,5,2,5,3,4,...,0,0,1,0,0,0,83,2023,1,4
157612,act1_245264,23,28,4,2,0,1,1,12,0,...,0,0,1,0,1,1,94,2023,3,16
157613,act1_9232,3,30,9,3,5,2,4,3,4,...,1,0,1,0,1,1,3,2023,1,26


In [59]:
df_all

,action_id,action_type,char_10_x,result,y_x,m_x,d_x,char_1,group_1,char_2,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,y_y,m_y,d_y
0,act2_1734928,2,5382,0,2023,8,26,1,3888,1,...,0,0,1,1,1,0,36,2021,6,29
1,act2_2434093,0,0,0,2022,9,27,1,3888,1,...,0,0,1,1,1,0,36,2021,6,29
2,act2_3404049,0,0,0,2022,9,27,1,3888,1,...,0,0,1,1,1,0,36,2021,6,29
3,act2_3651215,0,0,0,2023,8,4,1,3888,1,...,0,0,1,1,1,0,36,2021,6,29
4,act2_4109017,0,0,0,2023,8,26,1,3888,1,...,0,0,1,1,1,0,36,2021,6,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039671,act2_4668076,2,3057,1,2023,6,16,1,4258,2,...,1,0,1,1,1,1,95,2023,1,6
2039672,act2_4743548,2,871,1,2023,3,30,1,4258,2,...,1,0,1,1,1,1,95,2023,1,6
2039673,act2_536973,0,0,1,2023,1,19,1,4258,2,...,1,0,1,1,1,1,95,2023,1,6
2039674,act2_688656,2,1027,1,2023,5,2,1,4258,2,...,1,0,1,1,1,1,95,2023,1,6


### Готовим к обучению

In [10]:
from sklearn.model_selection import train_test_split

def make_test(df_type1):
    y = df_type1[['action_id', 'result']]
    X = df_type1.drop(['action_id', 'result'], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

def predict(X_train, X_test, y_train):
    RF = RandomForestClassifier(random_state=42)

    RF.fit(X_train, y_train)
    res = RF.predict(X_test)
    
    return res, RF

# Построение моделей

### Ммодель про type1

In [12]:
X_train_type1, X_test_type1, y_train_id_type1, y_test_id_type1 = make_test(df_type1)

# Оставляет только result
y_train_type1 = y_train_id_type1.drop('action_id', axis=1)
y_test_type1 = y_test_id_type1.drop('action_id', axis=1)


In [13]:
X_train_type1

,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,y_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,y_y,m_y,d_y
119832,22,27,0,2,5,1,2,12,17,2023,...,0,0,0,0,0,0,25,2022,11,4
7895,1,1,6,1,5,0,0,14,11,2022,...,0,0,0,0,0,1,48,2022,10,28
81980,1,11,6,1,5,0,0,14,11,2022,...,0,0,0,0,0,0,48,2022,8,9
7302,44,11,8,2,0,2,2,12,16,2023,...,0,0,0,0,0,0,26,2021,10,30
151204,33,11,6,2,0,2,0,12,1,2023,...,0,0,0,0,0,0,86,2023,2,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,11,27,4,1,5,0,0,14,17,2022,...,0,0,0,0,0,0,0,2022,10,5
103694,11,0,8,2,1,1,2,12,1,2023,...,0,0,0,0,0,0,0,2022,11,17
131932,44,11,6,0,5,0,0,16,3,2022,...,1,0,1,0,1,1,90,2022,9,29
146867,44,3,6,3,5,0,0,4,1,2022,...,0,0,0,0,0,0,47,2022,6,14


In [14]:
res_type1, model_type1 = predict(X_train_type1, X_test_type1, y_train_type1)
res_type1

<ipython-input-11-de5cb41611c5>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(X_train, y_train)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Модель про type_all
Делим на тестовую и валидационную

In [15]:
X_train_all, X_test_all, y_train_id_all, y_test_id_all = make_test(df_all)

# Оставляет только result

y_train_all = y_train_id_all.drop('action_id', axis=1)
y_test_all = y_test_id_all.drop('action_id', axis=1)



Предсказываем

In [16]:
res_all, model_all = predict(X_train_all, X_test_all, y_train_all)
res_all

<ipython-input-11-de5cb41611c5>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(X_train, y_train)


array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### Оценка результата

In [17]:
from sklearn.metrics import roc_auc_score

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


all_id = y_test_id_all.copy()
type1_id = y_test_id_type1.copy()

# Заносим результаты
all_id['preds'] = res_all
type1_id['preds'] = res_type1

# Объединяем
df_predicted = type1_id.append(all_id)

df_predicted

,action_id,result,preds
137803,act1_414249,0,0
71054,act1_148759,0,0
44916,act1_80633,0,0
155513,act1_166940,1,1
70751,act1_60187,0,0
...,...,...,...
1836546,act2_451228,0,0
1021679,act2_554285,1,1
2025131,act2_3593868,1,1
379924,act2_3777217,1,1


Рассчитываем показать ROC

In [18]:

goal = df_predicted['result']
prediction = df_predicted['preds']

scoce = roc_auc_score(goal, prediction)

In [21]:
scoce

0.9882721222444542

### Теперь обучим модели на полных тренировочных

**df_type1**

In [71]:
df_type1_base

,action_id,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,y_y,m_y,d_y
0,act1_9923,22,27,0,0,5,2,2,14,17,...,0,0,0,0,0,0,76,2022,8,26
1,act1_198174,29,2,6,0,5,0,0,12,0,...,0,0,0,0,0,0,0,2022,7,26
2,act1_214090,16,28,7,2,0,2,3,13,0,...,0,0,0,0,0,0,0,2022,7,26
3,act1_230588,11,11,4,2,4,1,1,12,11,...,0,0,0,0,0,0,0,2022,7,26
4,act1_271874,11,27,4,1,5,0,0,14,17,...,0,0,0,0,0,0,0,2022,7,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157610,act1_272254,3,0,6,3,5,0,0,10,8,...,1,0,1,0,1,0,50,2020,7,21
157611,act1_204921,3,30,9,3,5,2,5,3,4,...,0,0,1,0,0,0,83,2023,1,4
157612,act1_245264,23,28,4,2,0,1,1,12,0,...,0,0,1,0,1,1,94,2023,3,16
157613,act1_9232,3,30,9,3,5,2,4,3,4,...,1,0,1,0,1,1,3,2023,1,26


In [72]:
# df_type1

y = df_type1_base['result']
df_type1_base.drop(['action_id', 'result'], axis=1, inplace=True)
X = df_type1_base


In [73]:
model_type1.fit(X, y)

RandomForestClassifier(random_state=42)

In [74]:
model_type1.n_features_

54

**df_all**

In [75]:
# df_all

y = df_all_base['result']
df_all_base.drop(['action_id', 'result'], axis=1, inplace=True)
X = df_all_base

In [76]:
model_all.fit(X, y)

RandomForestClassifier(random_state=42)

In [77]:
model_all.n_features_

47

### Предскажем test

In [98]:
df_test = df_tt.copy()
df_persons = df_prs.copy()

In [99]:
df_test = separete_data(df_test)
df_persons = separete_data(df_persons, 1)

df_type1, df_all = concat_tables(df_test, df_persons)

df_type1 = convert_data(df_type1)
df_all = convert_data(df_all)

C:\Program Files (x86)\pit\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Модель type1

**type1**

In [100]:
type1_id = df_type1[['action_id']]
df_type1.drop('action_id', axis=1, inplace=True)

In [102]:
res_type1 = model_type1.predict(df_type1)

**all**

In [103]:
all_id = df_all[['action_id']]
df_all.drop('action_id', axis=1, inplace=True)

In [104]:
res_all = model_all.predict(df_all)

### Объединяем

In [106]:
# Заносим результаты
type1_id['preds'] = res_type1
all_id['preds'] = res_all

# Объединяем
df_predicted = type1_id.append(all_id)

df_predicted

<ipython-input-106-108873fc0526>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type1_id['preds'] = res_type1
<ipython-input-106-108873fc0526>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_id['preds'] = res_all


,action_id,preds
0,act1_249281,1
1,act1_240724,1
2,act1_83552,1
3,act1_218751,0
4,act1_383524,0
...,...,...
458590,act2_4367092,0
458591,act2_4404220,0
458592,act2_448830,0
458593,act2_450133,0


In [109]:
n = df_predicted.shape[0]
df_predicted.preds.value_counts()/n

1    0.581032
0    0.418968
Name: preds, dtype: float64

In [115]:
df_predicted = df_predicted.rename(columns={'preds':'result'})
df_predicted

,action_id,result
0,act1_249281,1
1,act1_240724,1
2,act1_83552,1
3,act1_218751,0
4,act1_383524,0
...,...,...
458590,act2_4367092,0
458591,act2_4404220,0
458592,act2_448830,0
458593,act2_450133,0


In [116]:
df_predicted.to_csv('two_models.csv', index=False)


In [120]:
df_tt

,person_id,action_id,date,action_type,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
Unnamed: 0,,,,,,,,,,,,,,
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,type 1,type 7,type 4,NaN
1,ppl_100004,act2_230855,2022-07-20,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 682
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 13,type 10,NaN
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,type 1,type 5,type 5,NaN
4,ppl_10001,act2_1043301,2022-10-15,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 3015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498682,ppl_99997,act2_4367092,2023-04-22,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1
498683,ppl_99997,act2_4404220,2022-11-12,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1
498684,ppl_99997,act2_448830,2022-08-02,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1


In [119]:
df_predicted.sort_values('action_id')

,action_id,result
19493,act1_1,1
6218,act1_100006,0
29085,act1_100050,1
4536,act1_100065,0
9547,act1_100068,0
...,...,...
251452,act2_999980,1
352912,act2_999982,1
440159,act2_999993,1
236761,act2_999995,0
